# Using batch deployments for image file processing

Importing the required libraries. This notebook requires:

- `azure-ai-ml`
- `mlflow`
- `azureml-mlflow`
- `numpy`
- `pandas`
- `pillow`
- `tensorflow`

In [2]:
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import BatchEndpoint, BatchDeployment, Model, AmlCompute, Data, BatchRetrySettings, CodeConfiguration, Environment
from azure.ai.ml.constants import AssetTypes, BatchDeploymentOutputAction
from azure.identity import DefaultAzureCredential

## Accessing the Azure Machine Learning workspace

In [3]:
subscription_id = "<subscription>"
resource_group = "<resource-group>"
workspace = "<workspace>"
   
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

Class RegistryOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


## About the model

Let's review how the model is build. The model was built using TensorFlow along with the RestNet architecture ([Identity Mappings in Deep Residual Networks](https://arxiv.org/abs/1603.05027)). This model has the following constrains that are important to keep in mind for deployment:

* In work with images of size 244x244 (tensors of `(224, 224, 3)`).
* It requires inputs to be scaled to the range `[0,1]`.

In [5]:
import tensorflow_hub as hub
import tensorflow as tf

model = tf.keras.Sequential(
    [
        hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5"),
    ])
model.build([None, None, None, 3])


Testing if the model works:

In [11]:
import PIL.Image as Image
import numpy as np

image_file = tf.keras.utils.get_file('image.jpeg','https://azuremlexampledata.blob.core.windows.net/data/imagenet/goldfish.JPEG')
img = Image.open(image_file).resize((244, 244))
img = np.array(img)/255.
batch_img = tf.expand_dims(img, axis=0)

Run the model:

In [12]:
pred = model.predict(batch_img)
pred_class = pred.argmax(axis=-1)

1/1 [==============================] - 1s 915ms/step


Getting the labels for ImageNet:

In [13]:
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://azuremlexampledata.blob.core.windows.net/data/imagenet/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

In [14]:
predicted_class_name = [imagenet_labels[predicted_class] for predicted_class in pred_class]
predicted_class_name

['goldfish']

Let's save this model locally:

In [12]:
model_local_path = 'imagenet-classifier/model'
model.save(model_local_path)

## Deploying the model in a batch endpoint

### Registering the model

We need to register the model in order to use it with Azure Machine Learning:

In [15]:
model_name = 'imagenet-classifier'


In [22]:
if not any(filter(lambda m : m.name == model_name, ml_client.models.list())):
    print(f'Model {model_name} is not registered. Creating...')
    model = ml_client.models.create_or_update(
        Model(name=model_name, path=model_local_path, type=AssetTypes.CUSTOM_MODEL)
    )

Let's get a reference to the model:

In [16]:
model = ml_client.models.get(name=model_name, label="latest")

### Creating an scoring script to work with the model

In [19]:
%%writefile imagenet-classifier/code/imagenet_scorer.py

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from os.path import basename
from PIL import Image
from tensorflow.keras.models import load_model


def init():
    global model
    global input_width
    global input_height

    # AZUREML_MODEL_DIR is an environment variable created during deployment
    model_path = os.path.join(os.environ["AZUREML_MODEL_DIR"], "model")

    # load the model
    model = load_model(model_path)
    input_width = 244
    input_height = 244

def run(mini_batch):
    results = []

    for image in mini_batch:
        data = Image.open(image).resize((input_width, input_height)) # Read and resize the image
        data = np.array(data)/255.0 # Normalize
        data_batch = tf.expand_dims(data, axis=0) # create a batch of size (1, 244, 244, 3)

        # perform inference
        pred = model.predict(data_batch)

        # Compute probabilities, classes and labels
        pred_prob = tf.math.reduce_max(tf.math.softmax(pred, axis=-1)).numpy()
        pred_class = tf.math.argmax(pred, axis=-1).numpy()

        results.append([basename(image), pred_class[0], pred_prob])

    return pd.DataFrame(results)

Overwriting imagenet-classifier/code/imagenet_scorer.py


### Creating the ednpoint

First, let's create the endpoint that is going to host the batch deployments. Remember that each endpoint can host multiple deployments at any time, however, only one of them is the default one:

In [20]:
endpoint_name = "imagenet-classifier-batch"
endpoint = BatchEndpoint(
    name=endpoint_name, 
    description="An batch service to perform imagenet image classification",
)

In [107]:
ml_client.batch_endpoints.begin_create_or_update(endpoint)

### Creating the compute

Batch endpoints can run on any Azure ML compute that already exists in the workspace. That means that multiple batch deployments can share the same compute infrastructure. In this example, we are going to work on an AzureML compute cluster called `cpu-cluster`. Let's verify the compute exists on the workspace or create it otherwise.

In [21]:
compute_name = "cpu-cluster"
if not any(filter(lambda m : m.name == compute_name, ml_client.compute.list())):
    print(f"Compute {compute_name} is not created. Creating...")
    compute_cluster = AmlCompute(name=compute_name, description="amlcompute", min_instances=0, max_instances=5)
    ml_client.begin_create_or_update(compute_cluster)

Compute may take time to be created. Let's wait for it:

In [22]:
from time import sleep

print("Waiting for compute", end="")
while ml_client.compute.get(name=compute_name).provisioning_state == "Creating":
    sleep(1)
    print(".", end="")

print(" [DONE]")

Waiting for compute [DONE]


### Creating the environment

Let's create the environment. In our case, our model runs on `TensorFlow`. Azure Machine Learning already has an environment with the required software installed, so we can reutilize this environment

In [31]:
environment = Environment(
    conda_file="./imagenet-classifier/environment/conda.yml",
    image="mcr.microsoft.com/azureml/tensorflow-2.4-ubuntu18.04-py37-cpu-inference:latest",
)

### Creating the deployment

Let's create a deployment under the given endpoint.

In [32]:
deployment = BatchDeployment(
    name="imagenet-classifier-resnetv2",
    description="A ResNetV2 model architecture for performing ImageNet classification in batch",
    endpoint_name=endpoint.name,
    model=model,
    environment=environment,
    code_configuration=CodeConfiguration(
        code="./imagenet-classifier/code/",
        scoring_script="imagenet_scorer.py",
    ),
    compute=compute_name,
    instance_count=2,
    max_concurrency_per_instance=1,
    mini_batch_size=10,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level="info",
)

In [33]:
ml_client.batch_deployments.begin_create_or_update(deployment)

### Setting the default deployment

Let's update the default deployment name in the endpoint:

In [30]:
endpoint = ml_client.batch_endpoints.get(endpoint_name)
endpoint.defaults.deployment_name = deployment.name
ml_client.batch_endpoints.begin_create_or_update(endpoint)

We can see the endpoint URL as follows:

In [31]:
endpoint.scoring_uri

'https://imagenet-classifier-batch.eastus.inference.ml.azure.com/jobs'

## Testing the endpoint

Once the deployment is created, it is ready to recieve jobs. Let's first register a data asset so we can run the job against it. This data asset is a folder containing 1000 images from the original ImageNet dataset. We are going to download it first and then create the data asset:

In [35]:
!wget https://azuremlexampledata.blob.core.windows.net/data/imagenet-1000.zip
!unzip imagenet-1000.zip -d /tmp/imagenet-1000

--2022-10-14 19:32:13--  https://azuremlexampledata.blob.core.windows.net/data/imagenet-1000.zip
Resolving azuremlexampledata.blob.core.windows.net (azuremlexampledata.blob.core.windows.net)... 20.209.0.229
Connecting to azuremlexampledata.blob.core.windows.net (azuremlexampledata.blob.core.windows.net)|20.209.0.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 108902090 (104M) [application/x-zip-compressed]
Saving to: ‘imagenet-1000.zip.1’

imagenet-1000.zip.1 100%[===================>] 103.86M  78.4MB/s    in 1.3s    

2022-10-14 19:32:16 (78.4 MB/s) - ‘imagenet-1000.zip.1’ saved [108902090/108902090]

Archive:  imagenet-1000.zip
  inflating: /tmp/imagenet-1000/n01440764_tench.JPEG  
  inflating: /tmp/imagenet-1000/n01443537_goldfish.JPEG  
  inflating: /tmp/imagenet-1000/n01484850_great_white_shark.JPEG  
  inflating: /tmp/imagenet-1000/n01491361_tiger_shark.JPEG  
  inflating: /tmp/imagenet-1000/n01494475_hammerhead.JPEG  
  inflating: /tmp/imagenet-1000

In [26]:
data_path = "/tmp/imagenet-1000"
dataset_name = "imagenet-sample-unlabeled"

imagenet_sample = Data(
    path=data_path,
    type=AssetTypes.URI_FOLDER,
    description="A sample of 1000 images from the original ImageNet dataset",
    name=dataset_name,
)

In [ ]:
ml_client.data.create_or_update(imagenet_sample)

Let's get a reference of the new data asset:

In [27]:
imagenet_sample = ml_client.data.get(name=dataset_name, label="latest")

Let's use this data as an input for the job:

In [28]:
input = Input(type=AssetTypes.URI_FOLDER, path=imagenet_sample.id)

In [34]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name,
    input=input
)

You can use the returned job object to check the status of the job:

In [35]:
ml_client.jobs.get(job.name)

Experiment,Name,Type,Status,Details Page
imagenet-classifier-batch,0c40ce8e-585a-4a86-90d2-654fae165cc5,pipeline,Preparing,Link to Azure Machine Learning studio


## High throughput deployments

In [13]:
%%writefile imagenet-classifier/code/imagenet_scorer_batch.py

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model


def init():
    global model
    global input_width
    global input_height

    # AZUREML_MODEL_DIR is an environment variable created during deployment
    model_path = os.path.join(os.environ["AZUREML_MODEL_DIR"], "model")

    # load the model
    model = load_model(model_path)
    input_width = 244
    input_height = 244

def decode_img(file_path):
    file = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(file, channels=3)
    img = tf.image.resize(img, [input_width, input_height])
    return img/255.

def run(mini_batch):
    images_ds = tf.data.Dataset.from_tensor_slices(mini_batch)
    images_ds = images_ds.map(decode_img).batch(64)

    # perform inference
    pred = model.predict(images_ds)

    # Compute probabilities, classes and labels
    pred_prob = tf.math.reduce_max(tf.math.softmax(pred, axis=-1)).numpy()
    pred_class = tf.math.argmax(pred, axis=-1).numpy()

    return pd.DataFrame([mini_batch, pred_prob, pred_class], columns=['file', 'probability', 'class'])

Overwriting imagenet-classifier/code/imagenet_scorer_batch.py


Let's use this new scoring script

In [ ]:
ht_deployment = BatchDeployment(
    name="imagenet-classifier-resnetv2-ht",
    description="A ResNetV2 model architecture for performing ImageNet classification in batch (High throughput)",
    endpoint_name=endpoint.name,
    model=model,
    environment="AzureML-tensorflow-2.4-ubuntu18.04-py37-cpu-inference@latest",
    code_configuration=CodeConfiguration(
        code="./imagenet-classifier/code/",
        scoring_script="imagenet_scorer_batch.py",
    ),
    compute=compute_name,
    instance_count=2,
    max_concurrency_per_instance=1,
    mini_batch_size=10,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level="info",
)

In [48]:
ml_client.batch_deployments.begin_create_or_update(ht_deployment)

This job/deployment uses curated environments. The syntax for using curated environments has changed to include azureml registry name: azureml:registries/azureml/environments//versions/ or azureml:registries/azureml/environments/labels/latest. Support for format you are using will be removed in future versions of the CLI and SDK. Learn more at aka.ms/curatedenv


Let's execute this specific deployment now:

In [ ]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name,
    deployment_name=ht_deployment,
    input=input
)

## Exploring the results

In [36]:
ml_client.jobs.download(name=job.name, download_path=".", output_name='score')

In [48]:
import pandas as pd

score = pd.read_csv("named-outputs/score/predictions.csv", header=None,  names=['file', 'class', 'probabilities'], sep=' ')
score['label'] = score['class'].apply(lambda pred: imagenet_labels[pred])
score

,file,class,probabilities,label
0,n02088094_Afghan_hound.JPEG,161,0.994745,Afghan hound
1,n02088238_basset.JPEG,162,0.999397,basset
2,n02088364_beagle.JPEG,165,0.366914,bluetick
3,n02088466_bloodhound.JPEG,164,0.926464,bloodhound
4,n02088632_bluetick.JPEG,165,0.989370,bluetick
...,...,...,...,...
805,n01986214_hermit_crab.JPEG,126,0.707628,hermit crab
806,n01990800_isopod.JPEG,127,0.999765,isopod
807,n02002556_white_stork.JPEG,128,0.999786,white stork
808,n02002724_black_stork.JPEG,129,0.997870,black stork


## Using MLflow with images

When working with MLflow models that processes images, it is important to take into account for all the preprocessing you model requires.

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf

model = tf.keras.Sequential(
    [
        tf.keras.layers.Resizing(244, 244, interpolation='bilinear', crop_to_aspect_ratio=False),
        tf.keras.layers.Rescaling(1/255.),
        hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5"),
        tf.keras.layers.Softmax(axis=-1)
    ])
model.build([None, None, None, 3])


Let's save this model in a local folder

In [ ]:
model_local_path = 'imagenet-classifier/model'
model.save(model_local_path)

We are going to include the labels for the predicted class in the directory so we can use them for inference:

In [ ]:
!wget https://azuremlexampledata.blob.core.windows.net/data/imagenet/ImageNetLabels.txt -d imagenet-classifier/model

Let's create a custom loader for the MLflow model:

In [82]:
%%writefile imagenet-classifier-mlflow/code/loader_module.py

class TfClassifier():
    def __init__(self, model_path: str, labels_path: str):
        import tensorflow as tf
        import numpy as np
        from tensorflow.keras.models import load_model
        
        self.model = load_model(model_path)
        self.imagenet_labels = np.array(open(labels_path).read().splitlines())

    def predict(self, data):
        preds = self.model.predict(data)

        pred_prob = tf.reduce_max(preds, axis=-1)
        pred_class = tf.argmax(preds, axis=-1)
        pred_label = [self.imagenet_labels[pred] for pred in pred_class]

        return pd.DataFrame([pred_class, pred_prob, pred_label], columns=['class', 'probability', 'label'])

def _load_pyfunc(data_path: str):
    import os

    model_path = os.path.abspath(data_path)
    labels_path = os.path.join(model_path, 'ImageNetLabels.txt')

    return TfClassifier(model_path, labels_path)

Overwriting imagenet-classifier-mlflow/code/loader_module.py


Indicating a signature for your model

In [ ]:
import numpy as np
import mlflow
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, TensorSpec

input_schema = Schema([
  TensorSpec(np.dtype(np.uint8), (-1, -1, -1, 3)),
])
signature = ModelSignature(inputs=input_schema)

Logging the model:

In [ ]:
mlflow.pyfunc.save_model('model', 
                        data_path='imagenet-classifier/model',
                        code_path=['imagenet-classifier-mlflow/code/module_loader.py'],
                        loader_module='module_loader',
                        signature=signature)